**EigenFaces**: PCA applied in Facial Recognition

In [1]:
from time import time
import logging
import pylab as pl
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import fetch_lfw_people
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.decomposition import PCA
from sklearn.svm import SVC

In [2]:
# Display progress logs
logging.basicConfig(level=logging.INFO, format='%(asctime)s %(message)s')

In [14]:
# Download the dataset and load it as numpy arrays
lfw_people = fetch_lfw_people(min_faces_per_person=20, resize=0.4)

# Look into the images arrays to find the shapes
n_samples, h, w = lfw_people.images.shape
np.random.seed(89)

# Define X
X = lfw_people.data
n_features = X.shape[1]

# Define ids of the people
y = lfw_people.target
target_names = lfw_people.target_names
n_classes = target_names.shape[0]

print("Total dataset size:")
print("n_samples: %d" % n_samples)
print("n_features: %d" % n_features)
print("n_classes: %d" % n_classes)

Total dataset size:
n_samples: 3023
n_features: 1850
n_classes: 62


In [15]:
# Split into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=89)

In [16]:
# Compute a PCA (eigenfaces) on the face dataset (treated as unlabeled dataset):
# unsupervised feature extraction / dimensionality reduction
n_components = 123

print("Extracting the top %d eigenfaces from %d faces" % (n_components, X_train.shape[0]))
t0 = time()
pca = PCA(n_components=n_components, svd_solver='randomized', whiten=True).fit(X_train)
print("done in %0.3fs" % (time() - t0))

eigenfaces = pca.components_.reshape((n_components, h, w))

print("Projecting the input data on the eigenfaces orthonormal basis")
t0 = time()
X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)
print("done in %0.3fs" % (time() - t0))

Extracting the top 123 eigenfaces from 2418 faces
done in 0.542s
Projecting the input data on the eigenfaces orthonormal basis
done in 0.046s


In [17]:
# Train a SVM classification model

print ("Fitting the classifier to the training set")
t0 = time()
param_grid = {
         'C': [1e3, 5e3, 1e4, 5e4, 1e5],
          'gamma': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.1],
          }
# for sklearn version 0.16 or prior, the class_weight parameter value is 'auto'
clf = GridSearchCV(SVC(kernel='rbf', class_weight='balanced'), param_grid)
clf = clf.fit(X_train_pca, y_train)
print ("done in %0.3fs" % (time() - t0))
print ("Best estimator found by grid search:")
print (clf.best_estimator_)

Fitting the classifier to the training set
done in 356.918s
Best estimator found by grid search:
SVC(C=1000.0, break_ties=False, cache_size=200, class_weight='balanced',
    coef0=0.0, decision_function_shape='ovr', degree=3, gamma=0.005,
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)


In [18]:
# Quantitative evaluation of the model quality on the test set

print ("Predicting the people names on the testing set")
t0 = time()
y_pred = clf.predict(X_test_pca)
print ("done in %0.3fs" % (time() - t0))

print (classification_report(y_test, y_pred, target_names=target_names))
print (confusion_matrix(y_test, y_pred, labels=range(n_classes)))


Predicting the people names on the testing set
done in 0.427s
                           precision    recall  f1-score   support

         Alejandro Toledo       0.75      0.67      0.71         9
             Alvaro Uribe       0.67      0.29      0.40         7
          Amelie Mauresmo       1.00      0.50      0.67         4
             Andre Agassi       0.17      0.33      0.22         3
           Angelina Jolie       0.50      0.33      0.40         3
             Ariel Sharon       0.59      0.81      0.68        16
    Arnold Schwarzenegger       1.00      0.20      0.33        10
     Atal Bihari Vajpayee       0.00      0.00      0.00         1
             Bill Clinton       0.75      0.50      0.60         6
             Carlos Menem       0.50      0.33      0.40         3
             Colin Powell       0.57      0.83      0.67        46
            David Beckham       0.50      0.25      0.33         8
          Donald Rumsfeld       0.68      0.71      0.70        21

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
